In [30]:
# !pip install financial-news-api
# from financial_news_api import FinancialNews

# api = FinancialNews(api_key="YOUR_API_KEY")
# articles = api.search(symbol="AAPL")

In [1]:
# !pip install gnews
!pip install lxml[html_clean] newspaper3k

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 19.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=5888578b46e0268eec96b2d75debd5872369f4326da5884b124648585fd4b516
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 4.1 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=c6e15a19a23376b05f2400791448a37445cfd4bcccdd158f80

In [49]:
import numpy as np
import pandas as pd
import yfinance as yf
from newspaper import Article
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

In [43]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: yiyanghkust/finbert-tone
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [4]:
ticker = yf.Ticker('RELIANCE.NS')

[{'id': '105392fb-c667-3e22-8938-0611af27dc06', 'content': {'id': '105392fb-c667-3e22-8938-0611af27dc06', 'contentType': 'STORY', 'title': 'US refiners struggle to absorb sudden surge in Venezuelan oil imports', 'description': '', 'summary': "HOUSTON/NEW YORK, Feb 3 (Reuters) - Oil refiners on the U. Gulf Coast are struggling to absorb a rapid surge in Venezuelan crude shipments since last month's flagship $2 billion supply deal between Caracas and Washington, pressuring prices and leaving some volumes unsold, according to traders and shipping data.", 'pubDate': '2026-02-03T17:21:29Z', 'displayTime': '2026-02-03T17:21:29Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/reuters.com/7ce93511e1f898da8ae52a221a09eb68', 'originalWidth': 800, 'originalHeight': 534, 'caption': 'A view of refineries in Freeport, Texas, U.S., June 23, 2025. REUTERS/Joel Angel Juarez', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.

In [5]:
news_lst = []
pub_date_lst = []
url_lst = []
for i in range(len(ticker.news)):
  # print(i)
  news_lst.append(ticker.news[i]['content']['summary'])
  pub_date_lst.append(ticker.news[i]['content']['pubDate'])
  url_lst.append(ticker.news[i]['content']['clickThroughUrl']['url'])

news_df = pd.DataFrame(news_lst)
news_df.columns = ['news']
news_df['pubDate'] = pub_date_lst
news_df['url'] = url_lst

In [7]:
# Create empty columns first
news_df['article_title'] = None
news_df['article_text'] = None

for idx, url in news_df['url'].items():
    try:
        article = Article(url)
        article.download()
        article.parse()

        news_df.at[idx, 'article_title'] = article.title
        news_df.at[idx, 'article_text'] = article.text

    except Exception as e:
        print(f"Failed at index {idx}: {e}")

In [37]:
def summarize_article_llama(article, tokenizer, model):
    # Handle empty or invalid text
    if not isinstance(article, str) or len(article.strip()) == 0:
        return ""

    # Chat-style prompt
    prompt = f"""
<|system|>
You are a financial news analyst.
Summarize the following article clearly and concisely.
Focus on key sentiment, and market impact.
<|user|>
{article}
<|assistant|>
"""

    try:
        # Tokenize
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(model.device)

        # Generate summary
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode
        summary = tokenizer.decode(
            output_ids[0],
            skip_special_tokens=True
        )

        # Remove prompt echo
        return summary.split("<|assistant|>")[-1].strip()

    except Exception as e:
        print(f"Summarization failed: {e}")
        return ""


# Apply to entire dataframe
news_df["news_summarized"] = news_df["article_text"].apply(
    lambda x: summarize_article_llama(x, tokenizer, model)
)

# Preview
news_df.head()


,news,pubDate,url,article_title,article_text,news_summarized,sentiment
0,"HOUSTON/NEW YORK, Feb 3 (Reuters) - Oil refine...",2026-02-03,https://finance.yahoo.com/news/us-refiners-str...,US refiners struggle to absorb sudden surge in...,By Marianna Parraga and Shariq Khan\n\nHOUSTON...,A report by Reuters highlights the challenges ...,neutral
1,Reliance Industries' fair value estimate has b...,2026-02-03,https://finance.yahoo.com/news/recent-developm...,What Recent Developments Mean For The Reliance...,Find winning stocks in any market cycle. Join ...,Simply Wall St's analysis of Reliance Industri...,neutral
2,"By Nidhi Verma SOUTH GOA, India, Jan 29 (Reute...",2026-01-29,https://finance.yahoo.com/news/indias-reliance...,"India's Reliance to buy up to 150,000 bpd of R...","By Nidhi Verma\n\nSOUTH GOA, India, Jan 29 (Re...","By Nidhi Verma,\n\nSOUTH GOA, India, January 2...",neutral
3,"By Nidhi Verma NEW DELHI/MOSCOW, Jan 21 (Reute...",2026-01-21,https://finance.yahoo.com/news/indias-reliance...,India's Reliance to buy sanctions-compliant Ru...,"By Nidhi Verma\n\nNEW DELHI/MOSCOW, Jan 21 (Re...","A financial news analyst, Nidhi Verma, summari...",neutral
4,Reliance Industries Ltd (WBO:RLI) reports a 10...,2026-01-21,https://finance.yahoo.com/news/reliance-indust...,Reliance Industries Ltd (WBO:RLI) Q3 2026 Earn...,Q : What were the main factors affecting Relia...,In the latest earnings report of Reliance Indu...,neutral


In [45]:
X = news_df['news_summarized'].to_list()
labels = {0:'neutral', 1:'positive', 2:'negative'}

sent_val = list()
scores = []
for x in X:
    inputs = tokenizer(x, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = finbert(**inputs)[0]
    # print(outputs.detach().numpy())

    val = labels[np.argmax(outputs.detach().numpy())]
    print(x, '----', val)
    print('#######################################################')
    sent_val.append(val)
    scores.append(outputs.tolist()[0])

A report by Reuters highlights the challenges faced by oil refiners on the US Gulf Coast as they struggle to absorb Venezuelan crude shipments after the country's flagging supply deal with Washington. The surge in Venezuelan crude has led to a shortage of supplies, with trading houses Vitol and Trafigura granted licenses to market and sell millions of barrels of Venezuelan oil. However, with Chevron also raising exports quickly, the trading companies are finding it hard to secure enough buyers among Gulf Coast refiners. "We're all facing this issue where there's more to place and not enough takers," one of the traders said, citing reluctance from U.S. refiners to buy Venezuelan crude. Some refiners are complaining that prices, albeit declining, remain high compared to competing Canadian heavy grades. Venezuelan heavy oil cargoes for delivery at the Gulf Coast are being offered at ⁠about $9.50 per barrel below benchmark Brent, versus discounts of between $6 and $7.50 per ‌barrel in mid 

In [46]:
scores_df = pd.DataFrame(scores, columns=['neutral', 'positive', 'negative'])

news_df['sentiment'] = sent_val
news_df['pubDate'] = pd.to_datetime(news_df['pubDate'])
news_df['pubDate'] = news_df['pubDate'].dt.strftime('%Y-%m-%d')
news_df_final = pd.concat([news_df, scores_df], axis = 1)
news_df_final

,news,pubDate,url,article_title,article_text,news_summarized,sentiment,neutral,positive,negative
0,"HOUSTON/NEW YORK, Feb 3 (Reuters) - Oil refine...",2026-02-03,https://finance.yahoo.com/news/us-refiners-str...,US refiners struggle to absorb sudden surge in...,By Marianna Parraga and Shariq Khan\n\nHOUSTON...,A report by Reuters highlights the challenges ...,negative,-3.851390,-1.205256,7.304321
1,Reliance Industries' fair value estimate has b...,2026-02-03,https://finance.yahoo.com/news/recent-developm...,What Recent Developments Mean For The Reliance...,Find winning stocks in any market cycle. Join ...,Simply Wall St's analysis of Reliance Industri...,neutral,6.111199,-2.908890,-5.418045
2,"By Nidhi Verma SOUTH GOA, India, Jan 29 (Reute...",2026-01-29,https://finance.yahoo.com/news/indias-reliance...,"India's Reliance to buy up to 150,000 bpd of R...","By Nidhi Verma\n\nSOUTH GOA, India, Jan 29 (Re...","By Nidhi Verma,\n\nSOUTH GOA, India, January 2...",neutral,5.321227,-4.235450,-3.173007
3,"By Nidhi Verma NEW DELHI/MOSCOW, Jan 21 (Reute...",2026-01-21,https://finance.yahoo.com/news/indias-reliance...,India's Reliance to buy sanctions-compliant Ru...,"By Nidhi Verma\n\nNEW DELHI/MOSCOW, Jan 21 (Re...","A financial news analyst, Nidhi Verma, summari...",neutral,2.604077,-4.492366,1.821155
4,Reliance Industries Ltd (WBO:RLI) reports a 10...,2026-01-21,https://finance.yahoo.com/news/reliance-indust...,Reliance Industries Ltd (WBO:RLI) Q3 2026 Earn...,Q : What were the main factors affecting Relia...,In the latest earnings report of Reliance Indu...,positive,-7.100163,11.702394,-6.168954
5,What Has Changed in the New Fair Value Estimat...,2026-01-20,https://finance.yahoo.com/news/story-behind-re...,How The Story Behind Reliance Industries (NSEI...,Make better investment decisions with Simply W...,Simply Wall St has updated its fair value esti...,neutral,7.450964,-3.944839,-5.882839
6,Jan 19 (Reuters) - Shares of India's Reliance ...,2026-01-19,https://finance.yahoo.com/news/indias-reliance...,India's Reliance falls after missing profit vi...,Jan 19 (Reuters) - Shares of India's Reliance ...,The article focuses on the financial performan...,negative,-4.793156,-3.142862,11.938461
7,Jan 16 (Reuters) - Indian billionaire Mukesh A...,2026-01-16,https://finance.yahoo.com/news/reliance-indust...,Billionaire Mukesh Ambani's Reliance misses qu...,Jan 16 (Reuters) - Indian billionaire Mukesh A...,The article focuses on Reliance Industries' fi...,negative,-4.436623,-3.423071,12.270184
8,"By Nidhi Verma NEW DELHI, Jan 15 (Reuters) - I...",2026-01-16,https://finance.yahoo.com/news/opec-regains-sh...,OPEC regains share in India as Russian oil imp...,"By Nidhi Verma\n\nNEW DELHI, Jan 15 (Reuters) ...","In December, India's Russian oil imports dropp...",negative,-3.959998,-1.358287,7.304627
9,Jan 12 (Reuters) - India's Reliance Industries...,2026-01-12,https://finance.yahoo.com/news/indias-reliance...,India's Reliance Industries says battery manuf...,Jan 12 (Reuters) - India's Reliance Industries...,Jan 12 (Reuters) - India's Reliance Industries...,neutral,2.086619,-0.066006,-4.045742
